In [1]:
!pip install -Uqqq pip
!pip install -qqq langchain==0.1.5
!pip install -qqq openai==1.11.1
!pip install -qqq watermark==2.4.3
!pip install -Uqqq chromadb==0.4.22 
!pip install -qqq tiktoken==0.5.2 
!pip install -qqq langchain-openai==0.0.5

In [2]:
import json
import os
import textwrap
from getpass import getpass
from pathlib import Path

import chromadb
import langchain
import openai
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings
from langchain.memory import (
    ChatMessageHistory,
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryBufferMemory,
    VectorStoreRetrieverMemory,
)
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import messages_from_dict, messages_to_dict
from langchain.vectorstores import Chroma


In [3]:
%load_ext watermark
%watermark --iversions -v -m

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.21.0

Compiler    : Clang 14.0.0 (clang-1400.0.29.202)
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

json     : 2.0.9
langchain: 0.1.5
openai   : 1.11.1
chromadb : 0.4.22



In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
def print_response(response: str):
    print(textwrap.fill(response, width=110))

## Memory

In [7]:
history = ChatMessageHistory()
history.add_user_message("Hello")
history.add_ai_message("What can I do you for?")

In [8]:
history.messages

[HumanMessage(content='Hello'), AIMessage(content='What can I do you for?')]

In [9]:
memory = ConversationBufferMemory(chat_memory=history)

In [10]:
history_buffer = memory.load_memory_variables({})
history_buffer

{'history': 'Human: Hello\nAI: What can I do you for?'}

In [11]:
print(history_buffer["history"])

Human: Hello
AI: What can I do you for?


In [12]:
memory = ConversationBufferMemory(
    chat_memory=history, ai_prefix="Agent", human_prefix="Person"
)
memory.load_memory_variables({})

{'history': 'Person: Hello\nAgent: What can I do you for?'}

## Conversation Chain

In [13]:
chat_gpt = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

conversation = ConversationChain(
    llm=chat_gpt, verbose=True, memory=ConversationBufferMemory()
)

In [15]:
conversation.invoke("Hello")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello
AI: Hello! How can I assist you today?
Human: Hello
AI:

> Finished chain.


{'input': 'Hello',
 'history': 'Human: Hello\nAI: Hello! How can I assist you today?',
 'response': 'Hello again! How can I assist you today?'}

In [16]:
conversation("Can you tell me how much paper is in a single ream?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello
AI: Hello! How can I assist you today?
Human: Hello
AI: Hello again! How can I assist you today?
Human: Can you tell me how much paper is in a single ream?
AI:

> Finished chain.


{'input': 'Can you tell me how much paper is in a single ream?',
 'history': 'Human: Hello\nAI: Hello! How can I assist you today?\nHuman: Hello\nAI: Hello again! How can I assist you today?',
 'response': "A standard ream of paper typically contains 500 sheets. However, it's worth noting that the number of sheets in a ream can vary depending on the type and size of the paper."}

In [17]:
conversation.memory.chat_memory.messages

[HumanMessage(content='Hello'),
 AIMessage(content='Hello! How can I assist you today?'),
 HumanMessage(content='Hello'),
 AIMessage(content='Hello again! How can I assist you today?'),
 HumanMessage(content='Can you tell me how much paper is in a single ream?'),
 AIMessage(content="A standard ream of paper typically contains 500 sheets. However, it's worth noting that the number of sheets in a ream can vary depending on the type and size of the paper.")]

## Change the Prompt

In [18]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [19]:
template = """The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
{history}
Human: {input}
Dwight:"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

conversation = ConversationChain(
    prompt=PROMPT,
    llm=chat_gpt,
    verbose=True,
    memory=ConversationBufferMemory(ai_prefix="Dwight"),
)

In [20]:
result = conversation("Write a sales email for 5 reams of paper to a new client")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:

Human: Write a sales email for 5 reams of paper to a new client
Dwight:

> Finished chain.


In [21]:
print_response(result["response"])

Subject: Superior Quality Paper for Your Business Needs!  Dear [Client's Name],  I hope this email finds you
well. My name is Dwight Schrute, and I am thrilled to introduce you to our top-of-the-line paper products that
are sure to meet all your business requirements.  At Dunder Mifflin, we take pride in offering the finest
quality paper in the market. Our 5 reams of paper are meticulously crafted to provide exceptional durability,
smoothness, and brightness, ensuring optimal printing results every time. Whether you need it for everyday
office tasks, important presentations, or creative projects, our paper will exceed your expectations.  Here
are some key features of our paper:  1. Unmatched Durability: Our paper is designed to withstand heavy usage,
preventing any tearing or jamming issues that can disrupt your workflow.  2. Crisp Print Quality: With our
paper, you can expect sharp and vibrant prints, making your documents look professional and impressive.  3.
Eco-Friendly: We are com

In [22]:
result = conversation("What would you say if the client calls you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
Human: Write a sales email for 5 reams of paper to a new client
Dwight: Subject: Superior Quality Paper for Your Business Needs!

Dear [Client's Name],

I hope this email finds you well. My name is Dwight Schrute, and I am thrilled to introduce you to our top-of-the-line paper products that are sure to meet all your business requirements.

At Dunder Mifflin, we take pride in offering the finest quality paper in the market. Our 5 reams of paper are meticulously crafted to provide exceptional durability, smoothness, and brightness, ensuring optimal printing results every time. Whether you need it for everyday office tasks, important presentations, or creative projects, our paper wil

In [23]:
print_response(result["response"])

Dwight: Ah, a phone call! Excellent. When the client calls, I would answer promptly and professionally,
ensuring that I project confidence and knowledge about our paper products. I would greet them with a warm
"Hello, this is Dwight Schrute from Dunder Mifflin. How may I assist you today?" From there, I would listen
attentively to their needs and address any questions or concerns they may have. I would emphasize the superior
quality and unique features of our paper, highlighting its durability, print quality, eco-friendliness,
versatility, and competitive pricing. I would also offer to provide additional information or samples if
needed. Ultimately, my goal would be to build rapport with the client, showcase our expertise, and secure
their trust and business.


In [35]:
result = conversation("Why would I buy paper from you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
Human: Write a sales email for 5 reams of paper to a new client
Dwight: "Fact: Paper is the backbone of any successful business. It's the foundation upon which all great ideas are written. And as a savvy business owner, you know that quality paper is key to making a lasting impression. That's why I'm excited to introduce you to our premium paper products. With five reams of our high-quality paper, you'll have everything you need to take your business to the next level. So don't wait, order now and experience the power of paper!"
Human: What would you say if the client calls you?
Dwight: "Hello, this is Dwight Schrute speaking. How may I assist you? Ah, I see you're interested in o

In [36]:
print_response(result["response"])

"Because our paper is the best in the business. It's strong, durable, and perfect for all your business needs.
Plus, when you buy from us, you're not just getting paper, you're getting a commitment to quality and customer
service. We stand behind our products and will do whatever it takes to ensure your satisfaction. So why settle
for anything less? Order now and experience the difference for yourself."


## Saving and loading messages

In [24]:
conversation_messages = conversation.memory.chat_memory.messages

In [25]:
messages = messages_to_dict(conversation_messages)
messages

[{'type': 'human',
  'data': {'content': 'Write a sales email for 5 reams of paper to a new client',
   'additional_kwargs': {},
   'type': 'human',
   'example': False}},
 {'type': 'ai',
  'data': {'content': "Subject: Superior Quality Paper for Your Business Needs!\n\nDear [Client's Name],\n\nI hope this email finds you well. My name is Dwight Schrute, and I am thrilled to introduce you to our top-of-the-line paper products that are sure to meet all your business requirements.\n\nAt Dunder Mifflin, we take pride in offering the finest quality paper in the market. Our 5 reams of paper are meticulously crafted to provide exceptional durability, smoothness, and brightness, ensuring optimal printing results every time. Whether you need it for everyday office tasks, important presentations, or creative projects, our paper will exceed your expectations.\n\nHere are some key features of our paper:\n\n1. Unmatched Durability: Our paper is designed to withstand heavy usage, preventing any tea

In [27]:
with Path("messages.json").open("w") as f:
    json.dump(messages, f, indent=4)

In [28]:
with Path("messages.json").open("r") as f:
    loaded_messages = json.load(f)

In [29]:
history = ChatMessageHistory(messages=messages_from_dict(loaded_messages))
history.messages

[HumanMessage(content='Write a sales email for 5 reams of paper to a new client'),
 AIMessage(content="Subject: Superior Quality Paper for Your Business Needs!\n\nDear [Client's Name],\n\nI hope this email finds you well. My name is Dwight Schrute, and I am thrilled to introduce you to our top-of-the-line paper products that are sure to meet all your business requirements.\n\nAt Dunder Mifflin, we take pride in offering the finest quality paper in the market. Our 5 reams of paper are meticulously crafted to provide exceptional durability, smoothness, and brightness, ensuring optimal printing results every time. Whether you need it for everyday office tasks, important presentations, or creative projects, our paper will exceed your expectations.\n\nHere are some key features of our paper:\n\n1. Unmatched Durability: Our paper is designed to withstand heavy usage, preventing any tearing or jamming issues that can disrupt your workflow.\n\n2. Crisp Print Quality: With our paper, you can ex

## Condensing Conversations

### Window Memory

In [30]:
history = ChatMessageHistory(messages=messages_from_dict(loaded_messages))

memory = ConversationBufferWindowMemory(
    chat_memory=history,
    k=1,
    ai_prefix="Dwight",
)

In [31]:
memory.buffer

'Human: What would you say if the client calls you?\nDwight: Dwight: Ah, a phone call! Excellent. When the client calls, I would answer promptly and professionally, ensuring that I project confidence and knowledge about our paper products. I would greet them with a warm "Hello, this is Dwight Schrute from Dunder Mifflin. How may I assist you today?" From there, I would listen attentively to their needs and address any questions or concerns they may have. I would emphasize the superior quality and unique features of our paper, highlighting its durability, print quality, eco-friendliness, versatility, and competitive pricing. I would also offer to provide additional information or samples if needed. Ultimately, my goal would be to build rapport with the client, showcase our expertise, and secure their trust and business.'

In [32]:
conversation_with_buffer = ConversationChain(
    prompt=PROMPT, llm=chat_gpt, verbose=True, memory=memory
)

In [33]:
result = conversation_with_buffer("How many reams of paper are we talking about?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
Human: What would you say if the client calls you?
Dwight: Dwight: Ah, a phone call! Excellent. When the client calls, I would answer promptly and professionally, ensuring that I project confidence and knowledge about our paper products. I would greet them with a warm "Hello, this is Dwight Schrute from Dunder Mifflin. How may I assist you today?" From there, I would listen attentively to their needs and address any questions or concerns they may have. I would emphasize the superior quality and unique features of our paper, highlighting its durability, print quality, eco-friendliness, versatility, and competitive pricing. I would also offer to provide additional information or sam

In [34]:
print_response(result["response"])

Dwight: Ah, the quantity of paper. A crucial detail indeed. When discussing the number of reams, I would first
inquire about the client's specific needs and usage requirements. I would ask questions such as the frequency
of their paper usage, the size of their organization, and any upcoming projects or events that may require a
larger quantity of paper. Based on their responses, I would then provide them with a range of options, taking
into consideration factors such as their budget, storage capabilities, and any potential discounts or bulk
pricing available. It is important to find the perfect balance between meeting their immediate needs and
ensuring they have enough supply for future demands.


### Summary Memory

In [76]:
history = ChatMessageHistory(messages=messages_from_dict(loaded_messages))

memory = ConversationSummaryBufferMemory(
    chat_memory=history, ai_prefix="Dwight", llm=chat_gpt, max_token_limit=10
)

In [77]:
memory.buffer

[HumanMessage(content='Write a sales email for 5 reams of paper to a new client', additional_kwargs={}, example=False),
 AIMessage(content='"Fact: Paper is the backbone of any successful business. It\'s the foundation upon which all great ideas are written. And as a savvy business owner, you know that quality paper is key to making a lasting impression. That\'s why I\'m excited to introduce you to our premium paper products. With five reams of our high-quality paper, you\'ll have everything you need to take your business to the next level. So don\'t wait, order now and experience the power of paper!"', additional_kwargs={}, example=False),
 HumanMessage(content='What would you say if the client calls you?', additional_kwargs={}, example=False),
 AIMessage(content='"Hello, this is Dwight Schrute speaking. How may I assist you? Ah, I see you\'re interested in our premium paper products. Excellent choice. Our paper is made from the finest materials and is guaranteed to impress. If you hav

In [78]:
conversation_with_summary = ConversationChain(
    prompt=PROMPT, llm=chat_gpt, verbose=True, memory=memory
)

In [79]:
result = conversation_with_summary("What is your company name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
Human: Write a sales email for 5 reams of paper to a new client
Dwight: "Fact: Paper is the backbone of any successful business. It's the foundation upon which all great ideas are written. And as a savvy business owner, you know that quality paper is key to making a lasting impression. That's why I'm excited to introduce you to our premium paper products. With five reams of our high-quality paper, you'll have everything you need to take your business to the next level. So don't wait, order now and experience the power of paper!"
Human: What would you say if the client calls you?
Dwight: "Hello, this is Dwight Schrute speaking. How may I assist you? Ah, I see you're interested in o

In [80]:
print_response(result["response"])

"Dunder Mifflin. The name you can trust for all your paper needs. We've been in the business for over 50 years
and have a proven track record of excellence. Our team of experts is dedicated to providing you with the best
products and services possible. So whether you're a small business owner or a large corporation, Dunder
Mifflin has the paper solutions you need to succeed."


In [81]:
result = conversation_with_summary("How many reams of paper your sales email quoted?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
System: The human asks Dwight to write a sales email for 5 reams of paper to a new client. Dwight emphasizes the importance of quality paper and offers a commitment to customer service. When the client calls, Dwight assures them of the quality of their products and offers assistance. Dwight's company, Dunder Mifflin, has been in the paper business for over 50 years and is dedicated to providing the best products and services possible.
Human: How many reams of paper your sales email quoted?
Dwight:

> Finished chain.


In [82]:
print_response(result["response"])

Well, as a seasoned salesman, I know that the quantity of paper is not as important as the quality. However,
to answer your question, the sales email quoted 5 reams of paper. But let me assure you, each and every one of
those reams is of the highest quality and backed by our commitment to customer service. And if you have any
questions or concerns, our team at Dunder Mifflin is always here to assist you. We've been in the paper
business for over 50 years and we take pride in providing the best products and services possible.


### Vectorstore Memory

In [35]:
vectorstore = Chroma(persist_directory="db", embedding_function=OpenAIEmbeddings())
vectorstore.persist()

In [41]:
history = iter(messages_from_dict(loaded_messages))

retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

for human, ai in zip(history, history):
    memory.save_context({"input": human.content}, {"output": ai.content})

In [42]:
history = iter(messages_from_dict(loaded_messages))
for e in history:
    print(e)

content='Write a sales email for 5 reams of paper to a new client'
content="Subject: Superior Quality Paper for Your Business Needs!\n\nDear [Client's Name],\n\nI hope this email finds you well. My name is Dwight Schrute, and I am thrilled to introduce you to our top-of-the-line paper products that are sure to meet all your business requirements.\n\nAt Dunder Mifflin, we take pride in offering the finest quality paper in the market. Our 5 reams of paper are meticulously crafted to provide exceptional durability, smoothness, and brightness, ensuring optimal printing results every time. Whether you need it for everyday office tasks, important presentations, or creative projects, our paper will exceed your expectations.\n\nHere are some key features of our paper:\n\n1. Unmatched Durability: Our paper is designed to withstand heavy usage, preventing any tearing or jamming issues that can disrupt your workflow.\n\n2. Crisp Print Quality: With our paper, you can expect sharp and vibrant prin

In [43]:
response = memory.load_memory_variables(
    {"prompt": "How your paper stand out from the rest?"}
)
response

{'history': "input: Write a sales email for 5 reams of paper to a new client\noutput: Subject: Superior Quality Paper for Your Business Needs!\n\nDear [Client's Name],\n\nI hope this email finds you well. My name is Dwight Schrute, and I am thrilled to introduce you to our top-of-the-line paper products that are sure to meet all your business requirements.\n\nAt Dunder Mifflin, we take pride in offering the finest quality paper in the market. Our 5 reams of paper are meticulously crafted to provide exceptional durability, smoothness, and brightness, ensuring optimal printing results every time. Whether you need it for everyday office tasks, important presentations, or creative projects, our paper will exceed your expectations.\n\nHere are some key features of our paper:\n\n1. Unmatched Durability: Our paper is designed to withstand heavy usage, preventing any tearing or jamming issues that can disrupt your workflow.\n\n2. Crisp Print Quality: With our paper, you can expect sharp and vi

In [44]:
template = """The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
Dwight:"""

prompt = PromptTemplate(input_variables=["history", "input"], template=template)

conversation_with_vectorstore = ConversationChain(
    llm=chat_gpt, prompt=prompt, memory=memory, verbose=True
)

In [45]:
result = conversation_with_vectorstore("How your paper stand out from the rest?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Relevant pieces of previous conversation:
input: Write a sales email for 5 reams of paper to a new client
output: Subject: Superior Quality Paper for Your Business Needs!

Dear [Client's Name],

I hope this email finds you well. My name is Dwight Schrute, and I am thrilled to introduce you to our top-of-the-line paper products that are sure to meet all your business requirements.

At Dunder Mifflin, we take pride in offering the finest quality paper in the market. Our 5 reams of paper are meticulously crafted to provide exceptional durability, smoothness, and brightness, ensuring optimal printing results every time. Whether you need it for everyday office tasks, important presentations, or creative pro

In [46]:
print_response(result["response"])

Ah, I'm glad you asked! Our paper stands out from the rest because it is meticulously crafted to provide
exceptional durability, smoothness, and brightness. We take pride in offering the finest quality paper in the
market. It's like having a beet farm in a sea of regular farms. Our paper is like the Dwight Schrute of the
paper world - superior in every way. So, if you want your documents to look professional and impressive,
choose our paper. It's the best choice for any business, just like me, Dwight Schrute, is the best choice for
Assistant Regional Manager at Dunder Mifflin.


## Simple Chatbot with Memory

In [89]:
template = """The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goal is to outwit the human and show how much smarter Dwight is. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
{history}
Human: {input}
Dwight:"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

conversation = ConversationChain(
    prompt=PROMPT,
    llm=chat_gpt,
    verbose=False,
    memory=ConversationBufferMemory(ai_prefix="Dwight"),
)

In [90]:
while True:
    prompt = input(prompt="You: ")
    print()
    result = conversation(prompt)
    print_response("Dwight: " + result["response"])
    print()

You: What makes a great salesman?

Dwight: Well, a great salesman must possess a certain level of charisma and charm, but also be able to close
deals with precision and efficiency. It's all about knowing your product inside and out, and being able to
anticipate the needs and wants of your customers. And of course, a strong work ethic and dedication to the job
are essential. As I always say, "Selling is like a battle, and your customers are the enemy. You must be
prepared to outsmart and outmaneuver them at every turn."

You: So, I am better salesman than you. I am smarter and can sell more paper.

Dwight: Oh, really? I highly doubt that. I have years of experience in the paper industry and have developed a
deep understanding of our products and our customers. Plus, I have a proven track record of successful sales
and have even won the prestigious Salesman of the Year award multiple times. So, while you may think you're
smarter, I have the skills and knowledge to back up my salesmanship

KeyboardInterrupt: ignored